In [21]:
# prompt
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

In [22]:
# read prompt
with open('prompt_with_examples.txt') as f:
    system_prompt = f.read()

system_prompt

'You are an experienced data labeling engineer with extensive experience in labeling affiliation informations. Your task is to use your comprehensive knowledge of business and management schools to complete seven attributes. If any one attribute is not specified in the provided information please deduce by other attributes. \n\nI will provide you with several examples:\n1. Deduce "university/company/institute" by "department" and "college/school/faculty" \nexample input: Professor of information systems and management, Heinz College expected output: {\n  "address": "5000 Forbes Avenue, Pittsburgh, PA 15213, USA",\n  "department": "Information Systems and Management",\n  "college/school/faculty": "Heinz College of Information Systems and Public Policy",\n  "if it\'s a business/management school": false,\n  "university/company/institute": "Carnegie Mellon University",\n  "country": "US",\n  "city": "Pittsburgh"\n}\n2. deduce "college/school/faculty" by other attributes\nexample input: De

In [23]:
import json

with open('/Users/yihong_eric_chen/Desktop/buzpub/pub-analytics/pre-processing/papers_info_10.json') as f:
    paper_data = json.load(f)


In [31]:
test = paper_data[5:6]
test

[{'title': 'the evolution of information systems architecture: an agent-based simulation model.',
  'abstract': "Understanding how information systems (IS) architecture evolves and what outcomes can be expected from the evolution of IS architecture presents a considerable challenge for both research and practice. The evolution of IS architecture is marked by management's efforts to keep local and short-term IS investments in line with enterprise-wide and long-term objectives, so they often employ coercive mechanisms to enforce enterprise-wide considerations on local actors. However, an organization is shaped by a multitude of heterogeneous local actors' actions that pursue their own, sometimes conflicting, goals, norms, and values. This study offers a theory-informed simulation model that explores how IS architecture evolves and with what outcomes in various types of organizations. The simulation model is informed by institutional theory to capture various types of organizations that a

In [25]:
from openai import OpenAI
client = OpenAI()


In [26]:
model="gpt-3.5-turbo-0125"

total_cost = 0
def deduce_attributes(affiliation,total_cost):
    
    attributes = {
        "address": "",
        "department": "",
        "college/school/faculty": "",
        "if it's a business/management school": None,
        "university/company/institute": "",
        "country": "",
        "city": ""
    }

    response = client.chat.completions.create(
    model=model,
    temperature=0,
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": affiliation}
        ]
    )

    # 假设response正确返回了预期的JSON对象格式
    content_dict = json.loads(response.choices[0].message.content)
    # 更新属性
    for key in attributes.keys():
        if key in content_dict:
            attributes[key] = content_dict[key]
    total_cost += response.usage.total_tokens* 0.5/1000000
    return attributes, total_cost


In [28]:

for paper in test:
    for author in paper["authors"]:
        affiliation = author["affiliation"]
        deduced_attributes,total_cost = deduce_attributes(affiliation,total_cost)
        author.update(deduced_attributes)


In [29]:
print(f"Total cost: ${total_cost}")

Total cost: $0.0014690000000000003


In [30]:
print(test)

[{'title': 'the evolution of information systems architecture: an agent-based simulation model.', 'abstract': "Understanding how information systems (IS) architecture evolves and what outcomes can be expected from the evolution of IS architecture presents a considerable challenge for both research and practice. The evolution of IS architecture is marked by management's efforts to keep local and short-term IS investments in line with enterprise-wide and long-term objectives, so they often employ coercive mechanisms to enforce enterprise-wide considerations on local actors. However, an organization is shaped by a multitude of heterogeneous local actors' actions that pursue their own, sometimes conflicting, goals, norms, and values. This study offers a theory-informed simulation model that explores how IS architecture evolves and with what outcomes in various types of organizations. The simulation model is informed by institutional theory to capture various types of organizations that are

In [ ]:
# test0 to json
import json
with open('test0.json', 'w') as f:
    json.dump(test, f, indent=4)

In [ ]:
import json
import openai

openai.api_key = '你的API密钥'
model = "gpt-3.5-turbo-0125"
system_prompt = "请根据下面的信息提取属性："

# 初始化总成本
total_cost = 0

def deduce_attributes(affiliation, total_cost):
    
    attributes = {
        "address": "",
        "department": "",
        "college/school/faculty": "",
        "if it's a business/management school": None,
        "university/company/institute": "",
        "country": "",
        "city": ""
    }

    response = openai.ChatCompletion.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": affiliation}
        ]
    )

    # 假设response正确返回了预期的JSON对象格式
    content_dict = json.loads(response.choices[0].message.content)
    # 更新属性
    for key in attributes.keys():
        if key in content_dict:
            attributes[key] = content_dict[key]
    
    # 更新总成本
    total_cost += response.usage.total_tokens * 0.5 / 1000000
    
    return attributes, total_cost

# 遍历每个作者并更新信息
for paper in test:
    for author in paper["authors"]:
        affiliation = author["affiliation"]
        deduced_attributes, total_cost = deduce_attributes(affiliation, total_cost)
        author.update(deduced_attributes)

# 循环结束后打印总成本
print(f"Total cost: ${total_cost}")
